# Spatter Benchmark Worflow

This notebook will guide you through the process of performing memory performance analysis using the Spatter benchmark. For more detail please refer to our [wiki](https://github.com/hpcgarage/spatter/wiki/Spatter-Benchmark-Workflow). As an example, we will use [Quicksilver](https://github.com/LLNL/Quicksilver) with the [All Absorb](https://github.com/LLNL/Quicksilver/blob/master/Examples/AllAbsorb/allAbsorb.inp) input.

### Installations

1) Install [Quicksilver](https://github.com/LLNL/Quicksilver)
    1) Build with OpenMP and debug symbols ON
2) Install [VTune](https://www.intel.com/content/www/us/en/developer/tools/oneapi/vtune-profiler-download.html)
3) Install [gs_patterns](https://github.com/lanl/gs_patterns)
    1) Make sure to also install [PIN](https://www.intel.com/content/www/us/en/developer/articles/tool/pin-a-dynamic-binary-instrumentation-tool.html) following the steps in the [pin_tracing](https://github.com/lanl/gs_patterns/tree/main/pin_tracing) folder

In [ ]:
export qs_dir="/nethome/jvalverde6/Quicksilver"
export notebook_dir="$(pwd)"
export gspatterns_dir="/nethome/jvalverde6/gs_patterns"
export pin_dir="/nethome/jvalverde6/gs_patterns/pin_dir"

### Identifying Hotspots

In order to identify memory hot spots, we will need to profile an execution of the application.

In [ ]:
module load vtune
vtune -collect hotspots "$qs_dir"/src/qs "$qs_dir"/Examples/AllAbsorb/allAbsorb.inp

Hotspot functions will be towards the end of the output, under **Top Hotspots**. 

Vtune might output some hotspot functions that look like:

1) func@0x1de24  
2) func@0x1dfd4

These are functions from an external library which are not traceable, therefore we will not focus on them. 

Functions that are traceable will usually have a friendlier name. We will trace the function **CycleTrackingFunction**.

Start by creating a file called **roi_funcs.txt**. This file should be located in the same directory where you call gs_patterns, and will contain the names of the functions you want to trace in your application, each on a separate line. 

You will find a **roi_funcs.txt** file already containing **CycleTrackingFunction** in the directory containing this notebook.

## Generating Traces

### Running Pin

Be advised running Pin might take a significant amount of time depending on the run time of your app, and how many functions you trace. 

In [ ]:
"$pin_dir"/pin -t "$pin_dir"/source/tools/ImemROI/obj-intel64/ImemROIThreads.so -- "$qs_dir"/src/qs "$qs_dir"/Examples/AllAbsorb/allAbsorb.inp

There should be an output file called **roitrace.00.CycleTrackingFunction.bin**. For the next step, we must **gzip** the file. Again, this step might take a significant amount of time depending on the size of the file. 

In [ ]:
gzip roitrace.00.CycleTrackingFunction.bin

### Running gs_patterns

This command will likely take the longest out of all. 

In [ ]:
"$gspatterns_dir"/build/gs_patterns roitrace.00.CycleTrackingFunction.bin.gz "$qs_dir"/src/qs

Once the above command is complete, you can run Spatter with the new **roitrace.00.CycleTrackingFunction.bin.json** file. Please refer to the [Getting Started](http://localhost:8080/notebooks/spatter/notebooks/GettingStarted.ipynb) notebook for more instructions on how to run Spatter. 

If you'd like to take a closer look at the pattern generated, please refer to the [Graphing Patterns]() notebook. 